In [ ]:
!pip install gensim==3.8.3
import gensim
gensim.__version__

In [ ]:
! wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip 

In [ ]:
! unzip /kaggle/working/mallet-2.0.8.zip 

In [ ]:
#!ls /kaggle/working/mallet-2.0.8

In [ ]:
#!conda update --force -y conda

In [ ]:
#!wget https://anaconda.org/conda-forge/gensim/3.8.3/download/win-64/gensim-3.8.3-py37h1834ac0_0.tar.bz2

In [ ]:
#!conda install /kaggle/working/gensim-3.8.3-py37h1834ac0_0.tar.bz2

In [ ]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from pprint import pprint
import spacy
import re
import sys

In [ ]:
import nltk; nltk.download('stopwords')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield (gensim.utils.simple_preprocess(str(sentence), deacc=True))


def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# orig_stdout = sys.stdout
# f = open('help.txt', 'w', encoding="utf-8")
# sys.stdout = f
# language = "en"
# stop_words = stopwords.words('english')
# data = pd.read_csv('/kaggle/input/datas/wine_part_new_1.csv', delimiter=";", low_memory=False)  # '/home/lily/Downloads/wines.csv'
# wine = data.iloc[1][1]

In [ ]:
#!sudo pip install git+https://github.com/statianas/pyLDAvis@master
#!sudo pip install pyLDAvis
#print(gensim.__version__)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
import os
import pickle

In [ ]:
os.environ.update({'MALLET_HOME':r'/kaggle/working/mallet-2.0.8/'})
#mallet_path = '\kaggle\input\mallet\mallet-2.0.8\bin\mallet' # update this path
mallet_path = '/kaggle/working/mallet-2.0.8/bin/mallet' # update this path
print(gensim.__version__)
#ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
def mallet_model(mallet_path, corpus, num_topics, id2word):
#     print("hello!")
    model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
    return model
#     print("hello!!")
#     with open('data.pickle', 'wb') as f_1:
#         pickle.dump(data, f_1)
#     print("hello!!!")
    

In [ ]:
import multiprocessing
import time
import threading

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
#         model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
#                                                         id2word=id2word,
#                                                         num_topics=num_topics,
#                                                             workers = 2,
#                                                         random_state=100,
#                                                         chunksize=100,
#                                                         passes=10,
#                                                         alpha='symmetric',
#                                                         per_word_topics=True)
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                        id2word=id2word,
                                                        num_topics=num_topics,
                                                        random_state=100,
                                                        chunksize=100,
                                                        passes=10,
                                                        alpha='auto',
                                                        iterations = 1500,
                                                        per_word_topics=True)
        #model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        #coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        #print("old", row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        #print("new", row)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

**

In [ ]:
#!pip install dataframe_image

In [ ]:
import threading
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from multiprocessing import Process, Queue
max_i = 0
Q = Queue()

In [ ]:
def give_optimal_model(model_list, coherence_values):
    max = -1
    global max_i
    optimal_model = model_list[0]
    for i in range(0, len(model_list)):
        if coherence_values[i] > max + 0.0001:
            max = coherence_values[i]
            optimal_model = model_list[i]
            max_i = i
            #print(coherence_values[i])
    #print(max)
    return optimal_model

In [ ]:
#import plotly.figure_factory as ff

In [ ]:
#!pip install -U kaleido

In [ ]:
!mkdir html

In [ ]:
!mkdir png

In [ ]:
!mkdir csv

In [ ]:
import signal

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException

signal.signal(signal.SIGALRM, timeout_handler)

In [ ]:
for num_f in range(3,4):
    orig_stdout = sys.stdout
    str1 = str("part_new_") + str(num_f) + str(".txt")
    f = open(str1, 'w', encoding="utf-8")
    sys.stdout = f
    language = "en"
    stop_words = stopwords.words('english')
    data = pd.read_csv("/kaggle/input/datas/wine_part_new_"+ str(num_f) + ".csv", delimiter=";", low_memory=False)  # '/home/lily/Downloads/wines.csv'
    data1 = data.values.tolist()
    wine = data.iloc[1][1]
    mas_notes = []
    flag = 0
    #print(type(wine))
    #print(wine, sep="\n")
    for row in data.itertuples():
        if (flag == 0) and (str(row[2]) != "Dogajolo Toscana Bianco 2017"): continue
        elif (flag == 0): 
            flag = 1
            wine = "Dogajolo Toscana Bianco 2017"

        #print(type(row[0]), type(row[1]), type(row[2]), type(row[3]), type(row[4]))
        # print(type(row[2]), "\t")
        #print("number =" + str(row[0]) + "\n", "year = " + str(row[1]) + "\n" + "wine_name=" + row[2] + "\n отзыв = " + row[3], "язык=" + row[4] + "\n")
        # break
        if (str(row[2]) == wine) and (row[4] == language):  # было row[2], в этот раз просто взяли без первого столбца
            mas_notes.append(row[3])
            continue
        if (str(row[2]) == wine): continue
        print("*********** ", wine, " ***********")
        # data = pd.read_csv("Wines1.csv", delimiter=";")

        data_words = list(sent_to_words(mas_notes))

        bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)  # higher threshold fewer phrases.
        trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  # 4

        # Faster way to get a sentence clubbed as a trigram/bigram
        bigram_mod = gensim.models.phrases.Phraser(bigram)
        trigram_mod = gensim.models.phrases.Phraser(trigram)

        # See trigram example
        # print(trigram_mod[bigram_mod[data_words[0]]])
        # Remove Stop Words
        data_words_nostops = remove_stopwords(data_words)

        # Form Bigrams
        data_words_bigrams = make_bigrams(data_words_nostops)
        # print(data_words_bigrams[0])


        # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
        # python3 -m spacy download en
        nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])  # en_core_web_md

        # Do lemmatization keeping only noun, adj, vb, adv
        data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

        # print(data_lemmatized[:1])
        # print(data_lemmatized[1:2])
        # print(data_lemmatized[2:3])
        # Почему эта штука удаляет нужное
        # Почему эта штука удаляет нужное

        id2word = corpora.Dictionary(data_lemmatized)
        texts = data_lemmatized
        # print(texts) test output
        corpus = [id2word.doc2bow(text) for text in texts]
        # print(len(corpus))
        # print([[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]])
        # print(corpus[:1])

        # Build LDA model 
        if (len(mas_notes) == 0):
            print("No reviews in English")
            wine = row[2]
            continue

        #sys.stdout = orig_stdout
        model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=5, limit=51, step=5)
        #sys.stdout = f
        limit=51
        start=5
        step=5
        x = range(start, limit, step)
        plt.cla()
        plt.clf()
        plt.plot(x, coherence_values)
        plt.xlabel("Num Topics")
        plt.ylabel("Coherence score")
        plt.legend(("coherence_values"), loc='best')
        plt.savefig(str('png/' + "plot" + "_"+ wine.replace(" ", "_")) +'.png')
        #plt.show()
        optimal_model = give_optimal_model(model_list, coherence_values) #тут по идее надо исправлять
        flag_alive = 0
#         for i in range(2):
#             p = Process(target=mallet_model, name="mallet_model", args=(mallet_path, corpus, 5+max_i*5, id2word,))
#             p.start()
#             p.run()
#             # c threads оно завершается, но видимо нельзя убить
#             #time.sleep(70)
#             p.join(40)
#             if not os.path.exists("/kaggle/working/data.pickle"):
#                 print("mallet_model is running... let's kill it...")
#                 # Terminate foo
#                 p.terminate()
#                 p.join()
#                 flag_alive = 1
#             else: break
#         for i in range(2):
        signal.alarm(120)    
        try:
            model = mallet_model(mallet_path, corpus, 5+max_i*5, id2word)
            signal.alarm(0)
            flag_alive = 0
            #break
        except TimeoutException:
            flag_alive = 1
            print('function terminated')
        if flag_alive == 1:
            wine = row[2]
            mas_notes = []
            time.sleep(10)
            continue
#         with open("/kaggle/working/data.pickle", 'rb') as f_1:
#             model = pickle.load(f_1)
        optimal_model = model

        #optimal_model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=5+max_i*5, id2word=id2word)
        #optimal_model = model

        #model_topics = optimal_model.show_topics(formatted=False)
        #pprint(optimal_model.print_topics(num_words=10))
        #------------------------------&-----------------------------
        df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=mas_notes) #тут

        df_dominant_topic = df_topic_sents_keywords.reset_index()
        df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
        #display(df_dominant_topic)

# Show #------------------------&--------------------------------------
        #print(df_dominant_topic.head(10))   
        sent_topics_sorteddf_mallet = pd.DataFrame()

        sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

        for i, grp in sent_topics_outdf_grpd:
            sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet,grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], axis=0)

# Reset Index    
        sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
        sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
        #print(sent_topics_sorteddf_mallet.head())
#-------------------------------&----------------------------
        topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
        topic_counts = topic_counts.sort_index()
        topic_counts = topic_counts.dropna()
        topic_counts_new = (topic_counts.reset_index())
        topic_counts_new = topic_counts_new.drop(columns=topic_counts_new.columns[0], axis=1)


# Percentage of Documents for Each Topic
        topic_contribution = (round(topic_counts/topic_counts.sum(), 4))
        topic_contribution = topic_contribution.reset_index()
        topic_contribution = topic_contribution.drop(columns=topic_contribution.columns[0], axis=1)

# Topic Number and Keywords
        #topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]
        #print(topic_num_keywords)
        sent_topics_sorteddf_mallet =  sent_topics_sorteddf_mallet.dropna()
        topic_num_keywords = sent_topics_sorteddf_mallet[['Topic_Num', 'Keywords']]

# Concatenate Column wise
        df_dominant_topics = pd.concat([topic_num_keywords, topic_counts_new, topic_contribution], axis=1)


# Change Column names
        df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
        #display(df_dominant_topics)

        #save_df_as_image(df_dominant_topics,str("table" + "_"+ wine.replace(" ", "_")) +'.png')
        df_dominant_topics.to_csv(str('csv/' + "csv_table" + "_"+ wine.replace(" ", "_") +'.csv'), sep = ";")
#             fig = ff.create_table(df_dominant_topics)
#             #fig.data[0]['columnwidth'] = [100, 2000]
#             fig.update_layout({"margin": {"l": 0, "r": 0, "b": 0, "t": 20}, autosize=True)
#             fig.write_image(str("table" + "_"+ wine.replace(" ", "_") +'.png'), scale=2)
#             my_data = genfromtxt(str("/kaggle/working/" + "csv_table" + "_"+ wine.replace(" ", "_") +'.csv'), delimiter=';')
#             im = Image.fromarray(my_data)
#             if im.mode != 'RGB':
#                 im = im.convert('RGB')
#             im.save("your_file.jpeg")
#         display(df_dominant_topics)
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                    id2word=id2word,
                                                    num_topics=len(df_dominant_topics),
                                                    random_state=100,
                                                    chunksize=100,
                                                    passes=10,
                                                    alpha='auto',
                                                    per_word_topics=True)

        doc_lda = lda_model[corpus]
        pyLDAvis.enable_notebook()
        vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
        vis.topic_coordinates['x'] = np.real(vis.topic_coordinates['x'])
        vis.topic_coordinates['y'] = np.real(vis.topic_coordinates['y'])
        pyLDAvis.save_html(vis, str('html/' + wine.replace(" ", "_") +'.html'))

                                                                    
                                                                
        wine = row[2]
        mas_notes = []
        

    
    print("*********** ", wine, " ***********")
    data_words = list(sent_to_words(mas_notes))

    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)  # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  # 4

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    # See trigram example
    # print(trigram_mod[bigram_mod[data_words[0]]])
    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)
    # print(data_words_bigrams[0])


    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])  # en_core_web_md

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    id2word = corpora.Dictionary(data_lemmatized)
    texts = data_lemmatized
    
    corpus = [id2word.doc2bow(text) for text in texts]
    
    # Build LDA model
    if (len(mas_notes) == 0):
        print("No reviews in English")
        wine = row[2]
        continue

#         lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                                     id2word=id2word,
#                                                     num_topics=topics,
#                                                     random_state=100,
#                                                     chunksize=100,
#                                                     passes=10,
#                                                     alpha='auto',
#                                                     per_word_topics=True)




#         doc_lda = lda_model[corpus]

    # Compute Coherence Score
#         coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
#         coherence_lda = coherence_model_lda.get_coherence()
    #print('\nCoherence Score: ', coherence_lda)
    #sys.stdout = orig_stdout
    model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=5, limit=51, step=5)
    #sys.stdout = f
    limit=51
    start=5
    step=5
    x = range(start, limit, step)
    plt.cla()
    plt.clf()
    plt.plot(x, coherence_values)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.savefig(str('png/' + "plot" + "_"+ wine.replace(" ", "_")) +'.png')
    #plt.show()
    optimal_model = give_optimal_model(model_list, coherence_values)
    #optimal_model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=5+max_i*5, id2word=id2word)
    flag_alive = 0
#     for i in range(2):
#         p = multiprocessing.Process(target=mallet_model, name="mallet_model", args=(mallet_path, corpus, 5+max_i*5, id2word,optimal_model,))
#         p.start()
#         p.join(35)
#         if len(arr) == 0:
#             print("mallet_model is running... let's kill it...")

#             # Terminate foo
#             p.terminate()
#             p.join()
#             flag_alive = 1
#         else: break
#     if flag_alive == 1:
#         wine = row[2]
#         mas_notes = []
#         continue
#     for i in range(2):
    signal.alarm(120)    
    try:
        model = mallet_model(mallet_path, corpus, 5+max_i*5, id2word)
        signal.alarm(0)
        flag_alive = 0
        #break
    except TimeoutException:
        flag_alive = 1
        print('function terminated')
    if flag_alive == 1:
        wine = row[2]
        mas_notes = []
        time.sleep(10)
        continue
    optimal_model = model

    #optimal_model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=5+max_i*5, id2word=id2word)
    #optimal_model = model
    #model_topics = optimal_model.show_topics(formatted=False)
    #pprint(optimal_model.print_topics(num_words=10))
    #------------------------------&-----------------------------
    df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=mas_notes) #тут

    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    # Show #------------------------&--------------------------------------

    sent_topics_sorteddf_mallet = pd.DataFrame()

    sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

    for i, grp in sent_topics_outdf_grpd:
        sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet,grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], axis=0)

# Reset Index    
    sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
    sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show

#-------------------------------&----------------------------
    topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
    topic_counts = topic_counts.sort_index()
    topic_counts = topic_counts.dropna()
    topic_counts_new = (topic_counts.reset_index())
    topic_counts_new = topic_counts_new.drop(columns=topic_counts_new.columns[0], axis=1)


# Percentage of Documents for Each Topic
    topic_contribution = (round(topic_counts/topic_counts.sum(), 4))
    topic_contribution = topic_contribution.reset_index()
    topic_contribution = topic_contribution.drop(columns=topic_contribution.columns[0], axis=1)

# Topic Number and Keywords

    sent_topics_sorteddf_mallet =  sent_topics_sorteddf_mallet.dropna()
    topic_num_keywords = sent_topics_sorteddf_mallet[['Topic_Num', 'Keywords']]

# Concatenate Column wise
    df_dominant_topics = pd.concat([topic_num_keywords, topic_counts_new, topic_contribution], axis=1)


# Change Column names
    df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
    #display(df_dominant_topics)
    #save_df_as_image(df_dominant_topics,str("table" + "_"+ wine.replace(" ", "_")) +'.png')
    df_dominant_topics.to_csv(str('csv/' + "csv_table" + "_"+ wine.replace(" ", "_") +'.csv'), sep = ";")
#         fig = ff.create_table(df_dominant_topics)
#         fig.update_layout(
#                     autosize=False,
#                     )
#         fig.write_image(str("table" + "_"+ wine.replace(" ", "_") +'.png'), scale=2)
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                    id2word=id2word,
                                                    num_topics=len(df_dominant_topics),
                                                    random_state=100,
                                                    chunksize=100,
                                                    passes=10,
                                                    alpha='auto',
                                                    per_word_topics=True)

    doc_lda = lda_model[corpus]
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    vis.topic_coordinates['x'] = np.real(vis.topic_coordinates['x'])
    vis.topic_coordinates['y'] = np.real(vis.topic_coordinates['y'])
    pyLDAvis.save_html(vis, str('html/' + wine.replace(" ", "_") +'.html'))

    sys.stdout = orig_stdout
    f.close()
